In [1]:
import pandas as pd
import ast
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import numpy as np
from importlib import resources
import io
import csv
import pgeocode
import re
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from tqdm import tqdm
tqdm.pandas()
import pytz
from numpy import nan
from hashlib import md5
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from money_parser import price_str
from datetime import datetime
from tqdm import tqdm
tqdm.pandas()
import pytz
from numpy import nan
from hashlib import md5



class data_prep:
    def __init__(self,df):
        self.df = df
#*************** City with Pincodes ***************************************#

        print("its working")
        print("avaliable funcations")
        print("insiatilaize a variable with data_prep and folllow the process")
        print("prep = data_prep(df) #initalising the module")
        print("df = prep.tansaction_code_hash() # for transaction code column name should be [order_id]")
        print("df = prep.email_hash() # for user_ code columns name should be [email]")
        print("df = prep.expand_items() # for expanding items columns name should have [items] in dict enclosed in a list")
        print("df = prep.product_seq() # for product sequence should have [product_name and quantity] ")
        print("df = prep.primary_key() # for primary_key should have columns [transaction_code and product_seq] ")
        print("df = prep.postal_code_city_map(eg.delivery_address) #  should have column containg address column for getting city and pincode [eg.user_address,delivery_address,seller_addres] ")
        return 

#************************************************************************************************************************************************#

    def tansaction_code_hash(self):
        """Hashing  transaction code"""
        self.df['transaction_code']=(self.df['order_id'].astype(str)+'hsr569**').progress_apply(lambda x: md5(x.encode()).hexdigest())
        return self.df
#***********************************************************************************************************************************************#
    def email_hash(self):
        """Hashing Email """
        self.df['user_code']=(self.df['email'].astype(str)+'hsr569**').progress_apply(lambda x: md5(x.encode()).hexdigest())
        return self.df

#***********************************************************************************************************************************************#    
    def expand_items(self):
        '''Raw data produced by parsers contain one order per row each order might contain more than one product. This function converts it to a different data, which contain one product per row. Input must be a pandas dataframe and output is a dataframe.
        
        Note:- The dictionaries in items column must not contain “None”. All dictionaries must be converted from string to python format (by using ast.literal_eval).
        '''
        self.df.dropna(subset=['items'],inplace=True)
        try:
            self.df['items'] = self.df['items'].apply(lambda x: ast.literal_eval(x))
            mn_lst=[]
            for oid,its in zip(self.df['order_id'].values,self.df['items'].values):
                data={}
                for i in its:
                    data=i
                    data['order_id']=oid
                    mn_lst.append(data)
            item_df=pd.DataFrame(mn_lst)
        except:
            self.df['items'] = self.df['items'].apply(lambda x: ast.literal_eval(x))
            mn_lst=[]
            for oid,its in zip(self.df['order_id'].values,self.df['items'].values):
                data={}
                for i in its:
                    data=i
                    data['order_id']=oid
                    mn_lst.append(data)
            item_df=pd.DataFrame(mn_lst)

        self.df = pd.merge(self.df,item_df[item_df.columns],on='order_id',how='left')

        return self.df

#******************************************************************************************************************************************************************#




    def prd_seq(self):
        '''Maps serial numbers to the products in each order and this is stored in column named product_seq. 
        Input must be a pandas dataframe and output is a dataframe.'''
        le = LabelEncoder()
        tc=le.fit_transform(self.df['order_id'].astype(str))
        pt = self.df['company'].apply(lambda x: x[:2].upper())
        self.df['transaction_code']=[p+'-'+"{:0>7}".format(t) for t,p in zip(tc,pt)]
        del tc,pt
        self.df.sort_values('transaction_code',inplace=True)
        gr1=self.df.groupby('transaction_code')
        sq=[]
        for tc,tc_df in tqdm(gr1):
            lt=list(range(len(tc_df['product_name'])))
            sq += lt
        self.df['product_seq']=sq
        self.df['product_seq'] += 1
        self.df['transaction_code']=(self.df['order_id'].astype(str)+'hsr569**').progress_apply(lambda x: md5(x.encode()).hexdigest())

        return self.df


#****************************************************************************************************************#



    def primary_key(self):
        '''combines sequence transaction code and product sequence'''
        self.df['primary_key'] = self.df['transaction_code']+"-"+self.df["product_seq"]       
        return self.df

#******************************************************************************************************************#
    def clean_ps(self,pn):
        '''Removes wrong postal codes. To be applied to column containing postal codes.'''
        if type(pn) != str:
            return None
        if (len(pn)==6 and pn.isdigit()):
            return pn
        else:
            return None
#*****************************************************************************************************************#
    def ext_pin(self,se):
        '''Extracts postal codes from address if it is available. To be applied to column containing address.'''
        pin=re.compile(r'\b\d{6}\b')
        try:
            match_obj=pin.findall(se)
            if not match_obj:
                match_obj=['']
        except TypeError:
            match_obj=['']
        return match_obj[-1]
#*******************************************************************************************************************#
    def postal_code_city_map(self,column_name):
        """cleaning postal code from address"""
        self.df[column_name] = self.df[column_name].astype(str).str.lower()
        self.df[column_name]=self.df[column_name].astype(str).str.lower().str.strip().apply(lambda x: " ".join(x.split()))
        self.df[column_name.rstrip("address")+"postal_code"]=self.df[column_name].apply(lambda x: self.ext_pin(x))
        self.df[column_name.rstrip("address")+"postal_code"]=self.df[column_name.rstrip("address")+"postal_code"].apply(lambda x: self.clean_ps(x))
        self.df[column_name.rstrip("address")+"postal_code"]=self.df[column_name.rstrip("address")+"postal_code"].astype(str).str.rstrip(".0")
        self.df.rename(columns={column_name.rstrip("address")+"postal_code":'postal_code'},inplace = True)
        self.df = pd.merge(self.df,self.city,on='postal_code',how='left')
        '''Maps postal code from internet if it is not mapped from csv file.'''
        ps_l=self.df[(self.df['postal_code'].notnull()) & (self.df['city'].isnull())]['postal_code'].unique()
        nomi = pgeocode.Nominatim('IN')
        pg=nomi.query_postal_code(ps_l)[['postal_code','county_name']]
        pg=pg.dropna(subset=['county_name']).reset_index(drop=True)
        self.df = pd.merge(self.df,pg,on = 'postal_code',how = 'left')
        self.df['county_name']=self.df['county_name'].str.lower()
        self.df['city'].fillna(self.df['county_name'],inplace=True)
        del self.df['county_name']
        self.df.rename(columns={'postal_code':column_name.rstrip("address")+"postal_code"},inplace = True)
        self.df.rename(columns={'city':column_name.rstrip("address")+"_city"},inplace = True)
        self.df= self.df.replace("None",np.nan)
        return self.df
#**********************************************************************************************************************#

#**************************************************************************************************************************#




In [2]:
# from vumonics_modules import data_prep

In [3]:
df = pd.read_csv("./complete_data_new_parser.csv")

C:\Users\SAGAR\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (30,31,43,45) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
prep=data_prep(df)
df = prep.tansaction_code_hash() # for transaction code column name should be [order_id]
df = prep.email_hash()
df = prep.expand_items()

its working
avaliable funcations
insiatilaize a variable with data_prep and folllow the process
prep = data_prep(df) #initalising the module
df = prep.tansaction_code_hash() # for transaction code column name should be [order_id]
df = prep.email_hash() # for user_ code columns name should be [email]
df = prep.expand_items() # for expanding items columns name should have [items] in dict enclosed in a list
df = prep.product_seq() # for product sequence should have [product_name and quantity] 
df = prep.primary_key() # for primary_key should have columns [transaction_code and product_seq] 
df = prep.postal_code_city_map(eg.delivery_address) #  should have column containg address column for getting city and pincode [eg.user_address,delivery_address,seller_addres] 


100%|██████████| 108870/108870 [00:00<00:00, 425894.51it/s]


In [5]:
df = prep.prd_seq()

100%|██████████| 216113/216113 [00:00<00:00, 317369.00it/s]


In [ ]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from money_parser import price_str
from datetime import datetime
from tqdm import tqdm
tqdm.pandas()
import pytz
from numpy import nan
from hashlib import md5
def prd_sq(df):
    le = LabelEncoder()
    tc=le.fit_transform(df['order_id'].astype(str))
    pt=df['company'].apply(lambda x: x[0:2].upper())
    df['transaction_code']=[p+'-'+"{:0>7}".format(t) for t,p in zip(tc,pt)]
    del tc,pt
    df.sort_values('transaction_code',inplace=True)
    gr1=df.groupby('transaction_code')
    sq=[]
    for tc,tc_df in tqdm(gr1):
        lt=list(range(len(tc_df['product_name'])))
        sq=sq+lt
    df['product_seq']=sq
    df['product_seq']=df['product_seq']+1
    df['transaction_code']=(df['order_id'].astype(str)+'hsr569**').progress_apply(lambda x: md5(x.encode()).hexdigest())

    return df

In [ ]:
 # for product sequence should have [product_name and quantity] 
df1 = prd_sq(df)
df1.columns